In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from cp_dataset import CPDataset, CPDataLoader
import argparse
import random
import torch
# Tạo đối tượng tùy chọn để tải dữ liệu
opt = argparse.Namespace(
    dataroot="data",  # dường dẫn đến thư mục
    datamode="train",  # Chế độ dữ liệu (train, test, ...)
    stage="GMM",  # GMM hoặc TOM
    data_list="train_pairs.txt",  # Tên tệp chứa danh sách cặp dữ liệu
    fine_width=192,
    fine_height=256,
    radius=5,
    grid_size=5,
    workers=1,  # Số tiểu trình được sử dụng để nạp dữ liệu
    batch_size=4,  # Thêm thuộc tính batch_size vào đây
    shuffle=True
)

# Tạo đối tượng dataset và data loader
dataset = CPDataset(opt)
data_loader = CPDataLoader(opt, dataset)

#random item
def random_integer():
    return random.randint(1, 1000)

# Sử dụng hàm để lấy một số nguyên ngẫu nhiên từ 1 đến 100
randomItem = random_integer()

# Lấy một mẫu từ data loader
sample = dataset.__getitem__(randomItem)  # Lấy mẫu đầu tiên, bạn có thể thay đổi chỉ số để lấy mẫu khác

# Hiển thị mẫu huấn luyện cụ thể
plt.figure(figsize=(12, 6))

# Hiển thị hình ảnh người
plt.subplot(1, 3, 1)
plt.imshow(sample['image'].permute(1, 2, 0))  # permute để chuyển đổi chiều của tensor
plt.title("Person Image")


cloth_tensor = sample['cloth']  # Lấy tensor của hình ảnh đồ án
gray_cloth_mask = torch.mean(cloth_tensor, dim=1, keepdim=True)
# Chuyển đổi tensor sang NumPy array
cloth_array = cloth_tensor.permute(1, 2, 0).numpy()

# Chuyển đổi NumPy array thành hình ảnh OpenCV
cloth_mask = (cloth_array * 255).astype(np.uint8)  # Điều chỉnh giá trị để chuyển sang 8-bit

# Chuyển sang không gian màu RGB
hsv = cv2.cvtColor(cloth_mask, cv2.COLOR_BGR2RGB)

# Tạo mask cho các pixel vượt qua ngưỡng và chuyển chúng thành màu đen
threshold = 250  # Ngưỡng màu trắng thuần, có thể điều chỉnh
mask = cv2.inRange(hsv, (threshold, threshold, threshold), (255, 255, 255))
cloth_mask[mask > threshold] = 0  # Chuyển các pixel vượt qua ngưỡng thành màu đen
cloth_mask[mask <= threshold] = 255  # Chuyển các pixel không vượt qua ngưỡng thành màu trắng

# Hiển thị hình ảnh đồ án và ảnh đen trắng
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(cloth_array)
plt.title("Cloth Image")


#hien thi cloth mark
plt.subplot(1, 3, 2)
plt.imshow(cloth_mask, cmap='gray')  # Hiển thị ảnh đen trắng
plt.title("Cloth Mask")


plt.tight_layout()
plt.show()


In [5]:
#chạy tensorboard
%load_ext tensorboard
%tensorboard --logdir tensorboard/GMM  --port 6060 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6060 (pid 3540), started 0:00:26 ago. (Use '!kill 3540' to kill it.)